In [ ]:
import math

import cv2
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from object_detection.core import standard_fields
from object_detection.utils import object_detection_evaluation, label_map_util, np_box_list_ops
from object_detection.utils import visualization_utils as vis_util
import time
import tensorflow as tf
import numpy as np
from tensorflow.contrib.predictor import predictor_factories

plt.rcParams['figure.figsize'] = (80, 60)

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
gpu_options = tf.GPUOptions(allow_growth=True)
tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

label_map_path = "data/label_map.pbtxt"
saved_model = "model/saved_model"

label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=7, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

predict_fn = predictor_factories.from_saved_model(saved_model)


In [ ]:
def darw_box(image,ground_truth_dict, output_dict, category_index):
    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        use_normalized_coordinates=False,
        line_thickness=3)

    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        ground_truth_dict['ground_truth_boxes'],
        ground_truth_dict['ground_truth_classes'],
        None,
        category_index,
        use_normalized_coordinates=False,
        line_thickness=3)
    return image

In [ ]:
def read_csv(image_dir, labels_csv):
    power_evaluator = object_detection_evaluation.ObjectDetectionEvaluator(categories=categories,
                                                                           evaluate_precision_recall=True)
    full_labels = pd.read_csv(labels_csv)
    filename = None
    for index, row in full_labels.iterrows():
        if filename == row["filename"]:
            continue
        else:
            filename = row["filename"]
        ground_truth_dict={}
        image = cv2.imread('{}/{}'.format(image_dir, filename))
        selected_value = full_labels[full_labels.filename == filename]
        ground_truth_dict["ground_truth_boxes"]=selected_value.values[:, 5:9]
        ground_truth_dict["ground_truth_classes"]=selected_value.values[:, 3]
        size_x = image.shape[1]
        size_y = image.shape[0]
        t0=time.time()
        output_dict = predict_fn({"inputs": np.expand_dims(image, axis=0)})
        print(f"{filename} size is {image.shape}, time is:{time.time()-t0}")
        index = np.where(output_dict['detection_scores'][0] > 0.5)
        output_dict['detection_classes'] = output_dict['detection_classes'][0][index].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0][index] * [size_y, size_x, size_y, size_x]
        output_dict['detection_scores'] = output_dict['detection_scores'][0][index]

        power_evaluator.add_single_ground_truth_image_info(filename, {
            standard_fields.InputDataFields.groundtruth_boxes: np.array(ground_truth_dict["ground_truth_boxes"], dtype=float),
            standard_fields.InputDataFields.groundtruth_classes: np.array(ground_truth_dict["ground_truth_classes"], dtype=int)
        })

        power_evaluator.add_single_detected_image_info(filename, {
            standard_fields.DetectionResultFields.detection_boxes: output_dict['detection_boxes'],
            standard_fields.DetectionResultFields.detection_scores: output_dict['detection_scores'],
            standard_fields.DetectionResultFields.detection_classes: output_dict['detection_classes'],
        })
        image=darw_box(image,ground_truth_dict, output_dict, category_index)
        plt.imshow(cv2.cvtColor(image,cv2.COLOR_BGR2RGB))
        plt.show()



    metrics = power_evaluator.evaluate()
    xml_list.append([""])
    xml_list.append([""])
    xml_list.append(['category_name', 'ap', 'precision', 'recall'])
    for c in categories:
        c_list = [c["name"]]
        precision = metrics["PerformanceByCategory/Precision@0.5IOU/b'{}'".format(c["name"])]
        ap = metrics["PerformanceByCategory/AP@0.5IOU/b'{}'".format(c["name"])]
        recall = metrics["PerformanceByCategory/Recall@0.5IOU/b'{}'".format(c["name"])]
        if math.isnan(ap):
            c_list.append("nan")
        else:
            c_list.append(ap)

        if isinstance(precision, float):
            c_list.append("nan")
        else:
            c_list.append(precision[-1])

        if isinstance(recall, float):
            c_list.append("nan")
        else:
            c_list.append(recall[-1])
        xml_list.append(c_list)
    xml_list.append(['Precision/mAP@0.5IOU', metrics['Precision/mAP@0.5IOU']])

In [ ]:
if __name__ == '__main__':
    read_csv("../南瑞项目素材收集-新版/01.大型机械/00.图片/02.湖北现场", "data/labels.csv")
    xml_df = pd.DataFrame(xml_list, columns=list(value_dict.keys()))
    xml_df.to_csv('big_mechanics.csv', index=None)